In [1]:
import json
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
data = sc.textFile('/home/hadoop/data/clean_data.json').map(json.loads)

In [16]:
label_count = data.flatMap(lambda x: x['Labels']). \
    map(lambda l: (l, 1)). \
    reduceByKey(int.__add__)
    
not_excl_l = label_count.filter(lambda lc: lc[1] > 129).map(lambda lc: lc[0]).collect()

In [19]:
len(not_excl_l)

625

In [20]:
not_excl_labels_br = sc.broadcast(set(not_excl_l))

In [21]:
def exclude_labels(doc):
    doc['Labels'] = [l for l in doc['Labels'] if l in not_excl_labels_br.value]
    return doc

In [22]:
ex_l = data.map(exclude_labels).filter(lambda x: x['Labels'])

In [23]:
word_count = ex_l.flatMap(lambda x: set(x['Text'])). \
    map(lambda w: (w, 1)). \
    reduceByKey(int.__add__)

In [9]:
wc_temp = word_count.filter(lambda wc: wc[1] > 1).map(lambda x: x[1])

In [32]:
wc = wc_temp.filter(lambda x: 999 < x).collect()

In [33]:
wc_temp.count()

207309

In [34]:
len(wc)

4207

In [35]:
plt.hist(wc, bins = 200)
plt.show()

In [36]:
excl_words = word_count.filter(lambda wc: wc[1] < 1000).map(lambda wc: wc[0]).collect()

In [37]:
excl_words_br = sc.broadcast(set(excl_words))

In [38]:
def exclude_words(doc):
    doc['Text'] = [w for w in doc['Text'] if w not in excl_words_br.value]
    return doc

In [39]:
ex_w = ex_l.map(exclude_words).filter(lambda x: len(x['Text']) > 49)

In [40]:
ex_w.count()

102797

## Checkpoint

In [41]:
clean_data = ex_w

In [29]:
clean_data.map(json.dumps).repartition(1).saveAsTextFile('hdfs://master:54310/nn_temp')

In [42]:
word_idx = clean_data.flatMap(lambda x: set(x['Text'])). \
    distinct(). \
    zipWithIndex(). \
    collectAsMap()
label_idx = clean_data.flatMap(lambda x: x['Labels']). \
    distinct(). \
    zipWithIndex(). \
    collectAsMap()
num_features = len(word_idx)

In [43]:
word_idx_br = sc.broadcast(word_idx)
label_idx_br = sc.broadcast(label_idx)

In [44]:
def vectorize_words(words: list):
    word_counts = Counter([word_idx_br.value[w] for w in words])
    return list(word_counts.items())

def vectorize_data(x: dict):
    features = vectorize_words(x['Text'])
    lables = [label_idx_br.value[l] for l in x['Labels']]
    return {'Labels':lables, 'Features':features}

In [45]:
vect_data = clean_data.map(vectorize_data)

## Checkpoint

In [46]:
with open('/home/hadoop/data/nn_ready/word_idx.json', 'w+') as fp:
    json.dump(word_idx, fp)
    
with open('/home/hadoop/data/nn_ready/label_idx.json', 'w+') as fp:
    json.dump(label_idx, fp)

In [47]:
json_dump = vect_data.map(lambda x: json.dumps(x) + '\n').collect()

In [48]:
with open('/home/hadoop/data/nn_ready/sparse_data.json', 'w+') as fp:
    fp.writelines(json_dump)

In [49]:
num_features

4181